# Projeto 1 - Ciência dos Dados

Nome: Henrique Rocha Bomfim

Nome: Nicholas Morales Balkins

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [92]:
import pandas as pd
import numpy as np
import os
import re

In [93]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

os.chdir("..\data")

Esperamos trabalhar no diretório
c:\Users\Henri\Documents\Insper\Ciência de Dados\Projeto 1\22-2a-cd-p1-grupo_henriquerb1\data


Carregando a base de dados com as notícias classificadas manualmente:

In [94]:
filename = 'dados.xlsx'

In [95]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Titulo,Descrição,Pagina,Target
0,Ucrânia oferece recompensa “generosa” para que...,A Ucrânia está de olho nas carteiras de cripto...,68,0
1,"Só 12,9% da oferta disponível de bitcoin no me...",Conforme o preço do bitcoin (BTC) continua sub...,100,2
2,Presidente do Paraguai veta lei que regula as ...,"O presidente do Paraguai, Mario Abdo Benítez, ...",4,2
3,Corretora de criptomoedas Coinbase vai entrar ...,"A Coinbase, a maior corretora de criptomoedas ...",5,0
4,Por que criptomoedas e esports estão cada vez ...,Quando a corretora de criptomoedas FTX anuncio...,13,1


In [96]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.tail(5)

,Titulo,Descrição,Pagina,Target
355,Corretora de criptomoedas de Hong Kong trava s...,Notícias de exchanges em queda continuam chega...,83,2
356,BNB cai 7% após EUA anunciarem investigação so...,A criptomoeda Binance Coin (BNB) mostra uma de...,34,2
357,PM de Goiás prende homem que vendia equipament...,Um homem de 21 anos foi preso no início da sem...,62,2
358,G20 fará proposta para regulamentar criptomoed...,"O Comitê de Estabilidade Financeira (ou FSB, n...",22,1
359,"Manhã cripto: Criptomoedas em queda, expectati...",O mercado de criptomoedas amanhece no vermelho...,62,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

Criptomoedas -> Se foi aderida para algum processo ou não, e se não se aplica à notícia.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [97]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub('(\s+)(de|em|do|para|e|da|que|com|a|no|das|dos|nos)(\s+)',' ',text_subbed)
    return text_subbed

train_titulo = train.Titulo
test_titulo = test.Titulo

train_0 = train.loc[train["Target"] == 0,:]
train_1 = train.loc[train["Target"] == 1,:]
train_2 = train.loc[train["Target"] == 2,:]

train_0_titulos = ''
for palavra in train_0.Titulo:
    train_0_titulos += palavra
train_0_titulos = cleanup(train_0_titulos).split()
train_0_titulos

train_1_titulos = ''
for palavra in train_1.Titulo:
    train_1_titulos += palavra
train_1_titulos = cleanup(train_1_titulos).split()

train_2_titulos = ''
for palavra in train_2.Titulo:
    train_2_titulos += palavra
train_2_titulos = cleanup(train_2_titulos).split()

serie_train_0_titulos = pd.Series(train_0_titulos)
serie_train_1_titulos = pd.Series(train_1_titulos)
serie_train_2_titulos = pd.Series(train_2_titulos)

In [98]:
tabela_absoluto_train_0 = serie_train_0_titulos.value_counts()
tabela_absoluto_train_1 = serie_train_1_titulos.value_counts()
tabela_absoluto_train_2 = serie_train_2_titulos.value_counts()

tabela_relativa_train_0 = serie_train_0_titulos.value_counts(True)
tabela_relativa_train_1 = serie_train_1_titulos.value_counts(True)
tabela_relativa_train_2 = serie_train_2_titulos.value_counts(True)
print(tabela_relativa_train_0,"\n")
print(tabela_relativa_train_1,"\n")
print(tabela_relativa_train_2)

criptomoedas         0.067028
o                    0.009364
sobre                0.008871
é                    0.008379
por                  0.008379
                       ...   
novas                0.000493
criptomoedasPreso    0.000493
PF                   0.000493
tráfico              0.000493
Plenário             0.000493
Length: 1197, dtype: float64 

criptomoedas             0.063260
mais                     0.015815
US                       0.015815
mercado                  0.015815
o                        0.013382
                           ...   
Libertadores             0.001217
patrocinar               0.001217
criptomoedasCorretora    0.001217
pagar                    0.001217
baixa                    0.001217
Length: 539, dtype: float64 

criptomoedas      0.060219
US                0.014599
mercado           0.014599
criptomoeda       0.012774
o                 0.010949
                    ...   
SingapuraPreço    0.001825
desativados       0.001825
eletrônicos       0

In [99]:
portugues = train_0_titulos + train_1_titulos + train_2_titulos
serie_portugues = pd.Series(portugues)
tabela_portugues_relativa = serie_portugues.value_counts
portugues_soma = tabela_absoluto_train_2.sum() + tabela_absoluto_train_1.sum() + tabela_absoluto_train_0.sum()
prob0 = tabela_absoluto_train_0.sum()/portugues_soma
prob1 = tabela_absoluto_train_1.sum()/portugues_soma
prob2 = tabela_absoluto_train_2.sum()/portugues_soma
print(prob0,"\n",prob1, '\n', prob2)
print(prob0 + prob1 + prob2)

0.596940276551927 
 0.2418358340688438 
 0.16122388937922918
1.0


In [100]:
prob_frase = tabela_portugues_relativa
frase = ''
a = 0
for i in range(len(train)): #len(train)
    frase = train_titulo[i]
    frase_lista = frase.split()

    probFraseDado0 = 1
    probFraseDado1 = 1
    probFraseDado2 = 1

    for palavra in frase_lista:
        if palavra in tabela_relativa_train_2:
            probFraseDado2 = probFraseDado2 * tabela_relativa_train_2[palavra]
      
        if palavra in tabela_relativa_train_1:
            probFraseDado1 = probFraseDado1 * tabela_relativa_train_1[palavra]

        if palavra in tabela_relativa_train_0:
            probFraseDado0 = probFraseDado0 * tabela_relativa_train_0[palavra]

probFraseDado0
probFraseDado1
probFraseDado2

6.09257092080452e-15

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [101]:
teste_titulo = test.Titulo
test_0 = test.loc[test["Target"] == 0,:]
test_1 = test.loc[test["Target"] == 1,:]
test_2 = test.loc[test["Target"] == 2,:]

test_0_titulos = ''
for palavra in test_0.Titulo:
    test_0_titulos += palavra
test_0_titulos = cleanup(test_0_titulos).split()

test_1_titulos = ''
for palavra in test_1.Titulo:
    test_1_titulos += palavra
test_1_titulos = cleanup(test_1_titulos).split()

test_2_titulos = ''
for palavra in test_2.Titulo:
    test_2_titulos += palavra
test_2_titulos = cleanup(test_2_titulos).split()

serie_test_0_titulos = pd.Series(test_0_titulos)
serie_test_1_titulos = pd.Series(test_1_titulos)
serie_test_2_titulos = pd.Series(test_2_titulos)

In [102]:
tabela_absoluto_test_0 = serie_test_0_titulos.value_counts()
tabela_absoluto_test_1 = serie_test_1_titulos.value_counts()
tabela_absoluto_test_2 = serie_test_2_titulos.value_counts()

tabela_relativa_test_0 = serie_test_0_titulos.value_counts(True)
tabela_relativa_test_1 = serie_test_1_titulos.value_counts(True)
tabela_relativa_test_2 = serie_test_2_titulos.value_counts(True)
print(tabela_relativa_test_0,"\n")
print(tabela_relativa_test_1,"\n")
print(tabela_relativa_test_2)

criptomoedas    0.059701
o               0.016681
na              0.010536
mais            0.009658
sobre           0.008780
                  ...   
estuda          0.000878
impedir         0.000878
Rússia          0.000878
fugir           0.000878
Binance         0.000878
Length: 760, dtype: float64 

criptomoedas      0.069641
na                0.014146
Bitcoin           0.013058
mercado           0.013058
o                 0.013058
                    ...   
metaverso         0.001088
20Coinbase        0.001088
todas             0.001088
semanaCoinbase    0.001088
outubro           0.001088
Length: 561, dtype: float64 

criptomoedas    0.066492
US              0.009624
criptomoeda     0.009624
mercado         0.008749
milhões         0.008749
                  ...   
acusa           0.000875
Celsius         0.000875
manipular       0.000875
CEL             0.000875
roubados        0.000875
Length: 727, dtype: float64


In [103]:
portugues = test_0_titulos + test_1_titulos + test_2_titulos
serie_portugues = pd.Series(portugues)
tabela_portugues_relativa = serie_portugues.value_counts
portugues_soma = tabela_absoluto_test_2.sum() + tabela_absoluto_test_1.sum() + tabela_absoluto_test_0.sum()
prob0 = tabela_absoluto_test_0.sum()/portugues_soma
prob1 = tabela_absoluto_test_1.sum()/portugues_soma
prob2 = tabela_absoluto_test_2.sum()/portugues_soma
print(prob0,"\n",prob1, '\n', prob2)
print(prob0 + prob1 + prob2)

0.3558263042799125 
 0.2870977819431428 
 0.3570759137769447
1.0


In [104]:
prob_frase = tabela_portugues_relativa

classifautolista = []

for i in range(len(test)):
    frase = train_titulo[i]
    frase_lista = frase.split()

    probFraseDado0 = 1
    probFraseDado1 = 1
    probFraseDado2 = 1

    for palavra in frase_lista:
        if palavra in tabela_relativa_test_2:
            probFraseDado2 = probFraseDado2 * tabela_relativa_test_2[palavra]
      
        if palavra in tabela_relativa_test_1:
            #probFraseDado1 = (probFraseDado1) * ((tabela_relativa_test_1[palavra] * len(serie_portugues) + 1)/(tabela_relativa_test_1[palavra]*tabela_absoluto_test_1.sum()))
            probFraseDado1 = probFraseDado1 * tabela_relativa_test_1[palavra]

        if palavra in tabela_relativa_test_0:
            #probFraseDado0 = (probFraseDado0) * ((tabela_relativa_test_0[palavra] * len(serie_portugues) + 1)/(tabela_relativa_test_0[palavra]*tabela_absoluto_test_0.sum()))
            probFraseDado0 = probFraseDado0 * tabela_relativa_test_0[palavra]

    prob0Frase = probFraseDado0*prob0
    prob1Frase = probFraseDado1*prob1
    prob2Frase = probFraseDado2*prob2

    if prob0Frase > prob1Frase and prob0Frase > prob2Frase:
        classifautolista.append(0)
    elif prob1Frase > prob0Frase and prob1Frase > prob2Frase:
        classifautolista.append(1)
    elif prob2Frase > prob0Frase and prob2Frase > prob1Frase:
        classifautolista.append(2)
    

print(classifautolista)
i = 0
acertos = 0

''' Estávamos fazendo as outras medições do item 4
quantos0manual = len(test.loc[test.Target == 0,:])
quantos1manual = len(test.loc[test.Target == 1,:])
quantos2manual = len(test.loc[test.Target == 2,:])'''

for j in classifautolista:
    if j == test.Target[i]:
        acertos += 1
    i += 1

Acuracia = acertos/len(test)
Acuracia

[0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 2, 0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 2, 0, 1, 2, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 2, 1, 2, 1, 1, 2, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 2, 0, 2, 0, 0, 0, 1, 1, 2, 2, 1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 2, 2, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 0, 1, 0, 0, 1, 

0.2972222222222222

___
### Concluindo

In [ ]:
#Nosso classificador não possui uma acurácia alta, porém planejamos continuar implementando ele para gerar melhores resultados. Além, acreditamos que a lógica do nosso código é condizente com o objetivo do projeto e que ele conseguiu fazer os cálculos de probabilidade e definir qual classificação daria para cada notícia.

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**